In [ ]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium
print ('Packages installed.')

In [66]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [67]:
import numpy as np
import pandas as pd
import geocoder
import requests 
from bs4 import BeautifulSoup 
import matplotlib.cm as cm
import matplotlib.colors as colors 
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


In [68]:
URL = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto"
r = requests.get (URL)  
soup = BeautifulSoup (r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 
print ('Page Scrapped.')

Page Scrapped.


In [69]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False
for row in soup.find_all ('td'):
    for cell in row:
        if cell.string and cell.string [0].isalpha () and len (cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string [1].isdigit ():
                    postalCodes.append (cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes [-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append (cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append (boroughs[-1])
                else:
                    neighborhoods.append (cell.string); 
                columnNum = 1          
print ('Data Collected.')

Data Collected.


In [70]:
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighbors = pd.DataFrame (columns=column_names)
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [72]:
lat_lng_coords = None
for data in range (0, len (postalCodes)-1):
    code = postalCodes [data]
    borough = boroughs [data]
    neighborhood_name = neighborhoods [data]
    g = geocoder.arcgis ('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng
    neighbors = neighbors.append ({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords [0],
                                   'Longitude': lat_lng_coords [1]}, ignore_index=True)
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,C01\n,Downtown,Harbourfront,43.64869,-79.38544
1,C02\n,The Annex,Yorkville,43.64869,-79.38544
2,C03\n,Forest Hill South,Oakwood–Vaughan,43.64869,-79.38544
3,C04\n,Bedford Park,Lawrence Manor,43.64869,-79.38544
4,C06\n,North York,Clanton Park,43.64869,-79.38544
5,C07\n,Willowdale West,Newtonbrook West,43.64869,-79.38544
6,C08\n,Cabbagetown,St. Lawrence Market,43.64869,-79.38544
7,C09\n,Moore Park,Rosedale,43.64869,-79.38544
8,C10\n,Davisville Village,Midtown Toronto,43.64869,-79.38544
9,C11\n,Leaside,Thorncliffe Park,43.64869,-79.38544


In [73]:
neighbors.shape

(62, 5)

In [74]:
VERSION = '20180605'
neighborhood_name = neighbors.loc [0, 'Neighborhood'] 
neighborhood_latitude = neighbors.loc [0, 'Latitude']
neighborhood_longitude = neighbors.loc [0, 'Longitude'] 
print ('Latitude and longitude values of {} are {}, {}.'.format (neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))
radius = 500
LIMIT = 100 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id=JGGBRN5XODTLZGJOMCSWIQMRH1JLGJKPSFR10XNB2R5U25GR&client_secret=KWRAMLK2HOJBQ2XLICLKXRU3M4HOCC1U2VG4Y4OPP5JF03QX&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'
results = requests.get (url).json()
results

Latitude and longitude values of Harbourfront are 43.648690000000045, -79.38543999999996.


{'meta': {'code': 200, 'requestId': '605e363efe5d453a8474a57a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'labe

In [75]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [76]:
from pandas.io.json import json_normalize
venues = results ['response']['groups'][0]['items']
nearby_venues = json_normalize (venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc [:, filtered_columns]
nearby_venues ['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [77]:
print ('{} venues were returned by Foursquare.'.format (nearby_venues.shape [0]))

4 venues were returned by Foursquare.


In [78]:
def getNearbyVenues (names, latitudes, longitudes):
    venues_list= []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print (name)
        results = requests.get (url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame ([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category'] 
    return (nearby_venues)

In [79]:
toronto_venues = getNearbyVenues (names=neighborhoods,
                                 latitudes=neighbors ['Latitude'],
                                 longitudes=neighbors ['Longitude'])

Harbourfront
Yorkville
Oakwood–Vaughan
Lawrence Manor
Clanton Park
Newtonbrook West
St. Lawrence Market
Rosedale
Midtown Toronto
Thorncliffe Park
C13

Willowdale East
Bayview Woods-Steeles
Danforth (Greektown)
Woodbine Corridor
East York
Dorset Park
Tam O'Shanter – Sullivan
Oakridge
E08

Woburn
Port Union (Centennial Scarborough)
Malvern
South Parkdale
Baby Point
Eglinton West
Glen Park
Humber Summit
Long Branch
W08

Richview (Willowridge)
Clairville


In [80]:
print (toronto_venues.shape)
toronto_venues.head()

(128, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.64869,-79.38544,Glen Manor Ravine,43.676821,-79.293942,Trail
1,Harbourfront,43.64869,-79.38544,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,Harbourfront,43.64869,-79.38544,Grover Pub and Grub,43.679181,-79.297215,Pub
3,Harbourfront,43.64869,-79.38544,Upper Beaches,43.680563,-79.292869,Neighborhood
4,Yorkville,43.64869,-79.38544,Glen Manor Ravine,43.676821,-79.293942,Trail


In [81]:
toronto_venues.groupby ('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Baby Point,4,4,4,4,4,4
Bayview Woods-Steeles,4,4,4,4,4,4
C13\n,4,4,4,4,4,4
Clairville,4,4,4,4,4,4
Clanton Park,4,4,4,4,4,4
Danforth (Greektown),4,4,4,4,4,4
Dorset Park,4,4,4,4,4,4
E08\n,4,4,4,4,4,4
East York,4,4,4,4,4,4


In [82]:
print ('There are {} uniques categories.'.format (len(toronto_venues ['Venue Category'].unique())))

There are 4 uniques categories.


In [83]:
toronto_onehot = pd.get_dummies (toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot ['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns [-1]] + list (toronto_onehot.columns [:-1])
toronto_onehot = toronto_onehot [fixed_columns]
toronto_onehot.head()

,Trail,Health Food Store,Neighborhood,Pub
0,1,0,Harbourfront,0
1,0,1,Harbourfront,0
2,0,0,Harbourfront,1
3,0,0,Harbourfront,0
4,1,0,Yorkville,0


In [84]:
toronto_onehot.shape

(128, 4)

In [85]:
toronto_grouped = toronto_onehot.groupby ('Neighborhood').mean ().reset_index ()
toronto_grouped

,Neighborhood,Trail,Health Food Store,Pub
0,Baby Point,0.25,0.25,0.25
1,Bayview Woods-Steeles,0.25,0.25,0.25
2,C13\n,0.25,0.25,0.25
3,Clairville,0.25,0.25,0.25
4,Clanton Park,0.25,0.25,0.25
5,Danforth (Greektown),0.25,0.25,0.25
6,Dorset Park,0.25,0.25,0.25
7,E08\n,0.25,0.25,0.25
8,East York,0.25,0.25,0.25
9,Eglinton West,0.25,0.25,0.25


In [86]:
toronto_grouped.shape

(32, 4)

In [89]:
num_top_venues = 5
for hood in toronto_grouped ['Neighborhood']:
    print ("_"+hood+"_")
    temp = toronto_grouped [toronto_grouped ['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc [1:]
    temp ['freq'] = temp ['freq'].astype (float)
    temp = temp.round({'freq': 2})
    print (temp.sort_values('freq', ascending=False).reset_index (drop=True).head(num_top_venues))
    print ('\n')

_Baby Point_
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


_Bayview Woods-Steeles_
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


_C13
_
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


_Clairville_
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


_Clanton Park_
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


_Danforth (Greektown)_
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


_Dorset Park_
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


_E08
_
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


_Eas

In [90]:
def return_most_common_venues (row, num_top_venues):
    row_categories = row.iloc [1:]
    row_categories_sorted = row_categories.sort_values (ascending=False)
    return row_categories_sorted.index.values [0:num_top_venues]

In [91]:
num_top_venues = 3
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange (num_top_venues):
    try:
        columns.append ('{}{} Most Common Venue'.format (ind+1, indicators[ind]))
    except:
        columns.append ('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame (columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
for ind in np.arange (toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc [ind, 1:] = return_most_common_venues (toronto_grouped.iloc [ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Baby Point,Pub,Health Food Store,Trail
1,Bayview Woods-Steeles,Pub,Health Food Store,Trail
2,C13\n,Pub,Health Food Store,Trail
3,Clairville,Pub,Health Food Store,Trail
4,Clanton Park,Pub,Health Food Store,Trail


In [92]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop ('Neighborhood', 1)
kmeans = KMeans (n_clusters=kclusters, random_state=0).fit (toronto_grouped_clustering)
kmeans.labels_[0:10]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [93]:
neighborhoods_venues_sorted.insert (0, 'Cluster Labels', kmeans.labels_)
toronto_merged = neighbors
toronto_merged = toronto_merged.join (neighborhoods_venues_sorted.set_index ('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,C01\n,Downtown,Harbourfront,43.64869,-79.38544,0,Pub,Health Food Store,Trail
1,C02\n,The Annex,Yorkville,43.64869,-79.38544,0,Pub,Health Food Store,Trail
2,C03\n,Forest Hill South,Oakwood–Vaughan,43.64869,-79.38544,0,Pub,Health Food Store,Trail
3,C04\n,Bedford Park,Lawrence Manor,43.64869,-79.38544,0,Pub,Health Food Store,Trail
4,C06\n,North York,Clanton Park,43.64869,-79.38544,0,Pub,Health Food Store,Trail


In [94]:
map_clusters = folium.Map (location= [43.67635739999999, -79.2930312], zoom_start=11)
x = np.arange (kclusters)
ys = [i + x + (i*x)**2 for i in range (kclusters)]
colors_array = cm.rainbow (np.linspace (0, 1, len(ys)))
rainbow = [colors.rgb2hex (i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip (toronto_merged ['Latitude'], toronto_merged ['Longitude'], toronto_merged ['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str (cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [95]:
toronto_merged.loc [toronto_merged ['Cluster Labels'] == 0, toronto_merged.columns [[1] + list (range (5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Downtown,0,Pub,Health Food Store,Trail
1,The Annex,0,Pub,Health Food Store,Trail
2,Forest Hill South,0,Pub,Health Food Store,Trail
3,Bedford Park,0,Pub,Health Food Store,Trail
4,North York,0,Pub,Health Food Store,Trail
5,Willowdale West,0,Pub,Health Food Store,Trail
6,Cabbagetown,0,Pub,Health Food Store,Trail
7,Moore Park,0,Pub,Health Food Store,Trail
8,Davisville Village,0,Pub,Health Food Store,Trail
9,Leaside,0,Pub,Health Food Store,Trail
